In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/KDT_ProjectManagement

/content/drive/MyDrive/Colab Notebooks/KDT_ProjectManagement


In [5]:
# 모듈 로딩
import pandas as pd
import numpy as np
import seaborn as sns
import re
import tensorflow as tf
#import string
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
nltk.download('all')

In [ ]:
!pip install konlpy

In [64]:
# KoNLPy Okt 임포트
from konlpy.tag import Okt

In [169]:
df_train = pd.read_csv("../data/관광데이터/train.csv")
df_test = pd.read_csv("../data/관광데이터/test.csv")

In [170]:
df_train.head()

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식


In [171]:
df_train = df_train.drop(['id','img_path'], axis=1)
df_train.head()

,overview,cat1,cat2,cat3
0,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식


In [172]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16986 entries, 0 to 16985
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   overview  16986 non-null  object
 1   cat1      16986 non-null  object
 2   cat2      16986 non-null  object
 3   cat3      16986 non-null  object
dtypes: object(4)
memory usage: 530.9+ KB


In [173]:
df_train['cat1'].value_counts()

인문(문화/예술/역사)    5614
음식              4912
레포츠             2611
자연              1719
숙박              1434
쇼핑               696
Name: cat1, dtype: int64

In [174]:
df_train['cat2'].value_counts()

음식점       4912
육상 레포츠    2288
자연관광지     1578
역사관광지     1571
숙박시설      1434
문화시설      1220
휴양관광지      889
체험관광지      805
쇼핑         696
축제         569
건축/조형물     328
수상 레포츠     277
관광자원       141
공연/행사      139
산업관광지       93
복합 레포츠      24
항공 레포츠      11
레포츠소개       11
Name: cat2, dtype: int64

In [175]:
df_train['cat3'].value_counts()

한식                3438
야영장,오토캠핑장         1558
바/까페               785
유적지/사적지            622
일반축제               549
                  ... 
MTB                  2
대중콘서트                2
인라인(실내 인라인 포함)       2
스카이다이빙               2
클래식음악회               2
Name: cat3, Length: 128, dtype: int64

In [176]:
df_train['cat3'].unique()

array(['항구/포구', '골프', '한식', '야영장,오토캠핑장', '섬', '일반축제', '유적지/사적지', '일식',
       '자연휴양림', '전시관', '관광단지', '모텔', '스키(보드) 렌탈샵', '바/까페', '채식전문점',
       '한옥스테이', '컨벤션', '성', '중식', '안보관광', '전문상가', '이색거리', '해수욕장', '5일장',
       '미술관/화랑', '공원', '상설시장', '박물관', '도서관', '공연장', '농.산.어촌 체험', '복합 레포츠',
       '펜션', '산', '기념탑/기념비/전망대', '기념관', '자동차경주', '자전거하이킹', '희귀동.식물',
       '유명건물', '수목원', '유원지', '사찰', '강', '자연생태관광지', '박람회', '문화전수시설', '계곡',
       '폭포', '서양식', '종교성지', '동상', '기타행사', '고택', '국립공원', '게스트하우스', '썰매장',
       '요트', '문화원', '래프팅', '승마', '공예,공방', '트래킹', '온천/욕장/스파', '수련시설',
       '해안절경', '패밀리레스토랑', '기암괴석', '기타', '홈스테이', '민물낚시', '콘도미니엄', '민박',
       '테마공원', '유람선/잠수함관광', '바다낚시', '수영', '백화점', '호수', '카약/카누', '외국문화원',
       '면세점', '다리/대교', '카지노', '분수', '동굴', '도립공원', '윈드서핑/제트스키', '특산물판매점',
       '유스호스텔', '고궁', '서비스드레지던스', '민속마을', '전통공연', '문', '식음료', '번지점프',
       '터널', '문화관광축제', '카트', '클럽', '발전소', '대중콘서트', 'MTB', '군립공원', '스케이트',
       '대형서점', '생가', '스카이다이빙', '영화관', '헹글라이딩/패러글라이딩', '스노쿨링/스킨스쿠버다이

In [177]:
# 중복 데이터 체크 및 삭제
df_train.duplicated().sum(), df_test.duplicated().sum()

(12, 0)

In [178]:
# 결측치 확인
df_train.isnull().sum()

overview    0
cat1        0
cat2        0
cat3        0
dtype: int64

In [179]:
# data, target 나누기
data = df_train["overview"]
#target = df_train["cat3"]
target = df_train['cat1']
print(data.shape, target.shape)

(16986,) (16986,)


In [180]:
# train, test ,validation 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(data, target, random_state = 42,stratify=target,test_size= 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state = 42,stratify=y_train,test_size= 0.2)

print(f'x_train.shape:{x_train.shape}, y_train.shape:{y_train.shape} ') 
print(f'x_test.shape:{x_test.shape}, y_test.shape:{y_test.shape}')
print(f'x_val.shape:{x_val.shape}, y_val.shape:{y_val.shape}')

x_train.shape:(10870,), y_train.shape:(10870,) 
x_test.shape:(3398,), y_test.shape:(3398,)
x_val.shape:(2718,), y_val.shape:(2718,)


In [182]:
x_train[0],y_train[:10]

KeyError: ignored

In [183]:
# target 인코딩
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

print(y_train)


[4 3 5 ... 3 0 4]


## [2] 텍스트 분류
---
- 토큰화
- 불용어 처리
- 정규식(영어로만 이루어진 단어)
- 소문자 변환
- 표제어 추출
- 문장 형태로 복원

In [138]:
okt = Okt()

In [139]:
# 텍스트 전처리 함수 
stop_words =['이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

def preprocessingText(sentence):
    words = []
    
    # 토큰화
    wordlist = okt.morphs(sentence) 
    for word in wordlist: 
        
        # 불용어 처리
        if word not in stop_words: 
            # 알파벳조합을 제외한 모든 문자 제거
            new_word = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '',word)
            
            words.append(new_word)
            #words.append(word)
            
    # 단어 -> 문장으로 복원
    return " ".join(words)

data = [preprocessingText(i) for i in data]

In [140]:
x_train = [preprocessingText(i) for i in x_train]
x_test = [preprocessingText(i) for i in x_test]
x_val = [preprocessingText(i) for i in x_val]

In [141]:
x_train[0:10],x_test[0],x_val[0]

(['양 문화원 은 지역 문화 를 창조 하고 시민 의 문화 삶 의 질 을 향상 시키고자 개원 하였다 지역 주민 의 문화 발전 을 위해 적극 인 활동 을 하고 있다',
  '태조 지방 의 중등교육 과 현 유 의 위패 를 봉안 배향 하고 지방 의 중등교육 과 지방 민 의 교화 를 위 하여 창건 되었다 정종 에 소실 되었다가 뒤 복원 하였으며 임진왜란 소실 되어 선조 대성전 을 명륜당 을 건립 하였다 대성전 과 명륜당 을 보수 하였으며 현 존 는 건물 로는 대성전 명륜당 동 재 서 재삼 문 교 직사 기타 부속 건물 있다 정면 칸 측면 칸 의 맞 배 지붕 으로 된 대성전 에는 성 송조 현 우리나라 현 의 위패 봉안 되어 있으며 대 성 성 문 선왕 이라 쓰인 공자 의 영정 은 조선 중기 송영 구 당나라 의 화가 오도현 의 작품 을 받아 온 이라 한다 조선시대 에는 국가 로부터 전답 과 노비 전적 을 지급 받아 교관 교생 을 가르쳤으나 현재 는 교육 기능 은 없어지고 봄 가을 석 을 봉행 하고 초하루 와 보름 에 분향 을 하고 있으며 전교 명과 장의 운영 을 담당 하고 있다 향교 에는 효종 부터 영조 까지의 역사 를 기록 서적 및 광해 조 부터 해오는 족자 있다 향교 의 대성전 은 전라북도 유형문화재 로 지정 되어 있다',
  '신라시대 보조선사 창건 하였다는 동양 보림 하나로 조계종 총 본 산 인 보림사 유명한 유치면 에 위치 하였다 보는 각도 에 따라 각기 모습 을 연출 하는 변화무쌍한 경치 와 계곡 과 계곡 사이 에 굽이쳐 흘러내리는 맑은 물 에서 세속 의 번뇌 를 잊고 편안한 휴식 을 취할 있는 곳 이다 길섶 까지 칡 덩굴 무성 하게 뻗어 있는 산길 을 가노라면 문명 에 오염 되지 않은 자연 그대로 의 산수 를 만날 있는 곳 이다 특히 보림사 에는 조상 의 얼 과 혼 담긴 귀중한 문화유산 국보 보물 소장 되어 있어 역사탐방 과 휴식 을 겸 할 있다 지역 특산 물 인 장흥 표고버섯 은 세계 인 명산 품 으로 표고버섯 전문 각종 요리 를 맛볼 있으며 산지 에서 신선한

In [142]:
# 텍스트 수치화
def transformText(data, num_words):
  tokenizer = Tokenizer(num_words= num_words) # num_words 
  tokenizer.fit_on_texts(data)
  seq_data = tokenizer.texts_to_sequences(data)
  
  # 사전 단어 총개수
  voca_word_num = len(tokenizer.word_index)
  return(seq_data,voca_word_num)

In [ ]:
seq_x_train,voca_word_num_x_train = transformText(x_train,100000)
seq_x_test,voca_word_num_x_test = transformText(x_test,100000)
seq_x_val,voca_word_num_x_val = transformText(x_val,100000)
print(f'단어사전 단어 개수: {voca_word_num_x_train}')

In [ ]:
# 단어 갯수 파악 함수
def checkLength(datas):

  # 기사 개당 단어 개수
  length=[len(data) for data in datas]
  
  # 히스토그램
  plt.figure(figsize=(12,8))
  plt.hist(length)
  plt.title(f'Max {max(length)}  Min {min(length)}  AVG {round(sum(length)/len(length),2)}')
  plt.xlabel('data length')
  plt.ylabel('data number')
  plt.show()
# return length
  
checkLength(seq_x_train)    

In [ ]:
# 패딩 설정:100
pad_length = 100
pad_x_train = pad_sequences(seq_x_train, maxlen= pad_length)
pad_x_test = pad_sequences(seq_x_test, maxlen= pad_length)
pad_x_val = pad_sequences(seq_x_val, maxlen= pad_length)

# 모델구축
---

In [ ]:
from tensorflow.keras.layers import SimpleRNN, GRU,Embedding, Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model

In [ ]:
EMB_DIM = 32                     # 임베딩 벡터 크기 
WORD_NUM = voca_word_num_x_train # 단어사전 수
HIDDEN_NODE = 1000                 # 은닉층 뉴런 수
INPUT_LENGH = pad_length         # 1문장의 토큰 수

In [ ]:
print(INPUT_LENGH,voca_word_num_x_train )

In [ ]:
model = Sequential() #EMB_DIM
model.add(Embedding(WORD_NUM,EMB_DIM, input_length=INPUT_LENGH))
model.add(GRU(HIDDEN_NODE,return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(HIDDEN_NODE))
model.add(Dropout(0.3))
model.add(GRU(HIDDEN_NODE))
model.add(Dropout(0.3))

model.add(Dense(128, activation='softmax'))

model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras.optimizers import Adam 
import tensorflow as tf
adam = tf.keras.optimizers.Adam(learning_rate =0.1)

# 하이퍼 파라미터 설정
---

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])

# patience
patience_earlystop = 5
patience_ReduceLROnPlateau = 2

# EarlyStop
earlystop = EarlyStopping(monitor='val_loss',
                          mode='min',
                          patience=patience_earlystop,
                          verbose=1)

# Learning Rate 조절 (변화량 0.5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=patience_ReduceLROnPlateau, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=1e-5)

In [218]:
epochs=30
batch_size=64

history = model.fit(pad_x_train,
                    y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data=(pad_x_val,y_val),
                    callbacks=[earlystop, learning_rate_reduction])

Epoch 1/30
170/170 [==============================] - 19s 94ms/step - loss: 1.8765 - acc: 0.2844 - val_loss: 1.6126 - val_acc: 0.3304 - lr: 0.0010
Epoch 2/30
170/170 [==============================] - 15s 88ms/step - loss: 1.6235 - acc: 0.3769 - val_loss: 1.5261 - val_acc: 0.4297 - lr: 0.0010
Epoch 3/30
170/170 [==============================] - 15s 88ms/step - loss: 1.4374 - acc: 0.4819 - val_loss: 1.7072 - val_acc: 0.3642 - lr: 0.0010
Epoch 4/30
170/170 [==============================] - ETA: 0s - loss: 1.3427 - acc: 0.5089
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
170/170 [==============================] - 15s 88ms/step - loss: 1.3427 - acc: 0.5089 - val_loss: 1.7586 - val_acc: 0.3966 - lr: 0.0010
Epoch 5/30
170/170 [==============================] - 15s 88ms/step - loss: 1.2778 - acc: 0.5352 - val_loss: 1.7615 - val_acc: 0.3915 - lr: 5.0000e-04
Epoch 6/30
170/170 [==============================] - ETA: 0s - loss: 1.2696 - acc: 0.5372
Epoch 6: Reduc

In [219]:
# 예측 확인

In [1]:
# 학습 히스토리 시각화
# epochs = range(1,30)

plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.show()



NameError: ignored

In [ ]:
model.evaluate(pad_x_test,y_test)